In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.linear_model import LogisticRegression
import pickle
import requests

In [3]:
!curl https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv -o AER_credit_card_data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73250  100 73250    0     0  1402k      0 --:--:-- --:--:-- --:--:-- 1402k


In [4]:
df = pd.read_csv("AER_credit_card_data.csv")

In [5]:
with open("model1.bin", "rb") as f:
    model = pickle.load(f)
with open("dv.bin", "rb") as f:
    dv = pickle.load(f)

In [8]:
'''
Question 3

Let's use these models!

    Write a script for loading these models with pickle
    Score this client:

{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

What's the probability that this client will get a credit card?

    0.162
    0.391
    0.601
    0.993

If you're getting errors when unpickling the files, check their checksum:

$ md5sum model1.bin dv.bin
3f57f3ebfdf57a9e1368dcd0f28a4a14  model1.bin
6b7cded86a52af7e81859647fa3a5c2e  dv.bin
'''
customer = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
x = dv.transform(customer)
print(x)
y_pred = model.predict_proba(x)
print(y_pred)

[[0.12     0.       1.       0.       0.001694]]
[[0.83786586 0.16213414]]


In [13]:
'''
Question 4

Now let's serve this model as a web service

    Install Flask and gunicorn (or waitress, if you're on Windows)
    Write Flask code for serving the model
    Now score this client using requests:

url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

What's the probability that this client will get a credit card?

    0.274
    0.484
    0.698
    0.928
'''
url = "http://localhost:9696/predict"
customer = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=customer).json()

{'credit_card': True, 'credit_card_prob': 0.9282218018527452}

In [6]:
'''
Question 6

Let's run your docker container!

After running it, score this client once again:

url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

What's the probability that this client will get a credit card now?

    0.289
    0.502
    0.769
    0.972
    
In dockerfile: EXPOSE 9696
               ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "predict:appFlask"]
Run docker:  docker run -it -p 9696:9696 mlzoomcamp bash
In docker if no entrypoint: gunicorn --bind=0.0.0.0:9696 predict:appFlask

'''
url = "http://localhost:9696/predict"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

{'credit_card': True, 'credit_card_prob': 0.7692649226628628}